In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from imp import reload
import torch.nn as nn
import torchvision

import sys
sys.path.append("..")
import vnn
import train_models
import nonvec_models
import vec_models
import dfa_util
reload(dfa_util)

<module 'dfa_util' from '../dfa_util.py'>

In [6]:
model = nonvec_models.make_cifar_nonvec_lc(False)
model = model.to(0)

model

DFAWrapper(
  (sequential): Sequential(
    (0): Local2d()
    (1): aReLU()
    (2): DFALayer()
    (3): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (4): Local2d()
    (5): aReLU()
    (6): DFALayer()
    (7): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (8): Local2d()
    (9): aReLU()
    (10): DFALayer()
    (11): Flatten()
    (12): Linear(in_features=512, out_features=512, bias=True)
    (13): aReLU()
    (14): DFALayer()
    (15): Linear(in_features=512, out_features=10, bias=True)
    (16): DFA()
  )
)

In [7]:
train_loader, test_loader = train_models.load_cifar()

Files already downloaded and verified
Files already downloaded and verified


In [8]:
opt = torch.optim.Adam(model.parameters(), lr=5e-4)
#flatten = nn.Flatten()
loss_fn = nn.CrossEntropyLoss(reduction="mean")

for epoch_idx in range(200):
    print(epoch_idx)
    for batch_idx, (data, labels) in enumerate(train_loader):
        opt.zero_grad()
        input = data.to(0) #flatten(data).to(0)
        output = model(input, learning_rule="df")
        loss = loss_fn(output, labels.to(0))
        loss.backward()
        opt.step()
        dfa_util.post_step_callback(model)

    


0
1
2
3
4
5


KeyboardInterrupt: 

In [10]:
num_correct = 0
total = 0
for batch_idx, (data, labels) in enumerate(test_loader):
    input = data.to(0) #flatten(data).to(0)
    with torch.no_grad():
        output = model(input, learning_rule="bp")
    num_correct += (output.argmax(dim=1).cpu() == labels).int().sum().item()
    total += len(data)
    
print(num_correct / total)

0.4086


In [18]:
model.sequential[15].weight.min()

tensor(0., device='cuda:0', grad_fn=<MinBackward1>)

In [42]:
phi = aReLU()



In [46]:
model = vec_models.make_cifar_vec_conv(True)
model = model.to(0)

"""
for layer in model:
    if hasattr(layer, 'weight'):
        layer.weight[:] = 0.
        print("ZERO")
"""

'\nfor layer in model:\n    if hasattr(layer, \'weight\'):\n        layer.weight[:] = 0.\n        print("ZERO")\n'

In [47]:
train_loader, test_loader = train_models.load_cifar()

Files already downloaded and verified
Files already downloaded and verified


In [48]:
opt = torch.optim.Adam(model.parameters(), lr=3e-4)
#flatten = nn.Flatten()
#loss_fn = nn.CrossEntropyLoss(reduction="mean")

for epoch_idx in range(2):
    print(epoch_idx)
    for batch_idx, (data, labels) in enumerate(train_loader):
        print(batch_idx)
        vnn.zero_grads(model)
        input = vnn.expand_input_conv(data, 10).to(0)
        output = model(input)[..., 0]
        vnn.set_model_grads(model, output, labels, reduction="mean", learning_rule="df")
        opt.step()
        vnn.post_step_callback(model)

    


0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276


In [49]:
num_correct = 0
total = 0
for batch_idx, (data, labels) in enumerate(train_loader):
    input = vnn.expand_input_conv(data, 10).to(0)
    output = model(input)[..., 0]
    num_correct += (output.argmax(dim=1).cpu() == labels).int().sum().item()
    total += len(data)
    
print(num_correct / total)

0.40616
